In [4]:
import pandas as pd
import time, datetime
import lolApi as lol
import lolModule as poc
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)
# pd.options.display.float_format = "{:,.2f}".format

# Data Type Json 파일 만들기

In [ ]:
# Reference data type
itemListDtype = dict(zip(["category", "position", "name_Kor", "name_Eng"], ["string", "string", "string", "string"]))
champInfoDtype = dict(zip(["index", "version", "id", "key", "name", "title", "blurb", "info", "image", "tags", "partype", "stats"], ["string", "string", "string", "int", "string", "string", "string", "string", "string", "string", "string", "string"]))
itemInfoDtype = dict(zip(["itemId", "name", "description", "colloq", "plaintext", "into", "image", "gold", "tags", "maps", "stats", "from", "depth", "effect", "stacks", "consumed", "inStore", "hideFromAll", "consumeOnFull", "specialRecipe", "requiredChampion"], ["int", "string", "string", "string", "string", "string", "string", "string", "string", "string", "string", "string", "float", "string", "float", "boolean", "boolean", "boolean", "boolean", "float", "string"]))
spellInfoDtype = dict(zip(["type", "version", "id", "name", "description", "tooltip", "maxrank", "cooldown", "cooldownBurn", "cost", "costBurn", "datavalues", "effect", "effectBurn", "vars", "key", "summonerLevel", "modes", "costType", "maxammo", "range", "rangeBurn", "image", "resource"], ["string", "string", "string", "string", "string", "string", "int", "string", "string", "string", "string", "string", "string", "string", "string", "int", "int", "string", "string", "int", "string", "string", "string", "string"]))
runeInfoDtype = dict(zip(["id", "key", "icon", "name", "longDesc", "shortDesc"], ["int", "string", "string", "string", "string", "string"]))
referenceDtype = dict(itemListDtype = itemListDtype, champInfoDtype = champInfoDtype, itemInfoDtype = itemInfoDtype, spellInfoDtype = spellInfoDtype, runeInfoDtype = runeInfoDtype)

# input data type
LeagueListDtype = dict(zip(["tier", "leagueId", "queue", "name", "summonerId", "summonerName", "leaguePoints", "rank", "wins", "losses", "veteran", "inactive", "freshBlood", "hotStreak", "miniSeries"], ["string", "string", "string", "string", "string", "string", "int", "string", "int", "int", "boolean", "boolean", "boolean", "boolean", "string"]))
MatchReferenceDtype = dict(zip(["platformId", "summonerName", "accountId", "gameId", "champion", "queue", "season", "matchDate", "timestamp", "role", "lane", "laneEdit"], ["string", "string", "string", "int", "int", "int", "int", "string", "int", "string", "string", "string"]))
ParticipantIdentityDtype = dict(zip(["gameId", "participantId", "platformId", "accountId", "summonerName", "summonerId", "currentPlatformId", "currentAccountId", "matchHistoryUri", "profileIcon"], ["int", "int", "string", "string", "string", "string", "string", "string", "string", "int"]))
ParticipantDtype = dict(zip(["gameId", "participantId", "teamId", "laneEdit", "championId", "spell1Id", "spell2Id", "win", "item0", "item1", "item2", "item3", "item4", "item5", "item6", "kills", "deaths", "assists", "largestKillingSpree", "largestMultiKill", "killingSprees", "longestTimeSpentLiving", "doubleKills", "tripleKills", "quadraKills", "pentaKills", "unrealKills", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "trueDamageDealt", "largestCriticalStrike", "totalDamageDealtToChampions", "magicDamageDealtToChampions", "physicalDamageDealtToChampions", "trueDamageDealtToChampions", "totalHeal", "totalUnitsHealed", "damageSelfMitigated", "damageDealtToObjectives", "damageDealtToTurrets", "visionScore", "timeCCingOthers", "totalDamageTaken", "magicalDamageTaken", "physicalDamageTaken", "trueDamageTaken", "goldEarned", "goldSpent", "turretKills", "inhibitorKills", "totalMinionsKilled", "neutralMinionsKilled", "neutralMinionsKilledTeamJungle", "neutralMinionsKilledEnemyJungle", "totalTimeCrowdControlDealt", "champLevel", "visionWardsBoughtInGame", "sightWardsBoughtInGame", "wardsPlaced", "wardsKilled", "firstBloodKill", "firstBloodAssist", "firstTowerKill", "firstTowerAssist", "firstInhibitorKill", "firstInhibitorAssist", "perk0", "perk0Var1", "perk0Var2", "perk0Var3", "perk1", "perk1Var1", "perk1Var2", "perk1Var3", "perk2", "perk2Var1", "perk2Var2", "perk2Var3", "perk3", "perk3Var1", "perk3Var2", "perk3Var3", "perk4", "perk4Var1", "perk4Var2", "perk4Var3", "perk5", "perk5Var1", "perk5Var2", "perk5Var3", "perkPrimaryStyle", "perkSubStyle", "statPerk0", "statPerk1", "statPerk2", "role", "lane"], ["int", "int", "int", "string", "int", "int", "int", "boolean", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "boolean", "boolean", "boolean", "boolean", "boolean", "boolean", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "string", "string"]))
MatchParticipantFrameDtype = dict(zip(["gameId" , "timestamp" , "participantId" , "position_y" , "position_x" , "currentGold" , "totalGold" , "level" , "xp" , "minionsKilled" , "jungleMinionsKilled" , "position_xEdit" , "position_yEdit"], ["int", "int", "int", "float", "float", "int", "int", "int", "int", "int", "int", "float", "float"]))
MatchEventDtype = dict(zip(["gameId", "type", "timestamp", "participantId", "itemId", "skillSlot", "levelUpType", "wardType", "creatorId", "position", "killerId", "victimId", "assistingParticipantIds", "afterId", "beforeId", "monsterType", "teamId", "buildingType", "laneType", "towerType", "monsterSubType"], ["int", "string", "int", "float", "float", "float", "string", "string", "float", "string", "float", "float", "string", "float", "float", "string", "float", "string", "string", "string", "string"]))
inputDtype = dict(LeagueListDtype = LeagueListDtype, MatchReferenceDtype = MatchReferenceDtype, ParticipantIdentityDtype = ParticipantIdentityDtype, ParticipantDtype = ParticipantDtype, MatchParticipantFrameDtype = MatchParticipantFrameDtype, MatchEventDtype = MatchEventDtype)

# output data type
champLaneStatsDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "champLevel", "kills", "deaths", "assists", "largestKillingSpree", "largestMultiKill", "firstBloodKill", "firstBloodAssist", "firstTowerKill", "firstTowerAssist", "firstInhibitorKill", "firstInhibitorAssist", "killingSprees", "doubleKills", "tripleKills", "quadraKills", "pentaKills", "turretKills", "inhibitorKills", "longestTimeSpentLiving", "totalTimeCrowdControlDealt", "timeCCingOthers", "totalDamageDealtToChampions", "physicalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "trueDamageDealt", "largestCriticalStrike", "damageDealtToObjectives", "damageDealtToTurrets", "totalHeal", "totalUnitsHealed", "damageSelfMitigated", "totalDamageTaken", "physicalDamageTaken", "magicalDamageTaken", "trueDamageTaken", "visionScore", "wardsPlaced", "wardsKilled", "visionWardsBoughtInGame", "sightWardsBoughtInGame", "goldEarned", "goldSpent", "totalMinionsKilled", "neutralMinionsKilled", "neutralMinionsKilledTeamJungle", "neutralMinionsKilledEnemyJungle", "totalGold10Diff", "xp10Diff", "minionsKilled10Diff", "jungleMinionsKilled10Diff", "totalGold15Diff", "xp15Diff", "minionsKilled15Diff", "jungleMinionsKilled15Diff", "totalGold20Diff", "xp20Diff", "minionsKilled20Diff", "jungleMinionsKilled20Diff", "totalGold25Diff", "xp25Diff", "minionsKilled25Diff", "jungleMinionsKilled25Diff", "totalGold30Diff", "xp30Diff", "minionsKilled30Diff", "jungleMinionsKilled30Diff", "totalGold10SupCount", "xp10SupCount", "minionsKilled10SupCount", "jungleMinionsKilled10SupCount", "totalGold15SupCount", "xp15SupCount", "minionsKilled15SupCount", "jungleMinionsKilled15SupCount", "totalGold20SupCount", "xp20SupCount", "minionsKilled20SupCount", "jungleMinionsKilled20SupCount", "totalGold25SupCount", "xp25SupCount", "minionsKilled25SupCount", "jungleMinionsKilled25SupCount", "totalGold30SupCount", "xp30SupCount", "minionsKilled30SupCount", "jungleMinionsKilled30SupCount", "totalGold10SupProp", "xp10SupProp", "minionsKilled10SupProp", "jungleMinionsKilled10SupProp", "totalGold15SupProp", "xp15SupProp", "minionsKilled15SupProp", "jungleMinionsKilled15SupProp", "totalGold20SupProp", "xp20SupProp", "minionsKilled20SupProp", "jungleMinionsKilled20SupProp", "totalGold25SupProp", "xp25SupProp", "minionsKilled25SupProp", "jungleMinionsKilled25SupProp", "totalGold30SupProp", "xp30SupProp", "minionsKilled30SupProp", "jungleMinionsKilled30SupProp"], ["string", "int", "int", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float"]))
champPurchasedDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "itemId", "itemWinRate", "itemCount", "itemProp"], ["string", "int", "int", "float", "float", "int", "float", "int", "float"]))
champRuneDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "perk0", "perk1", "perk2", "perk3", "perk4", "perk5", "perkWinRate", "perkCount", "perkProp"], ["string", "int", "int", "float", "float", "int", "int", "int", "int", "int", "int", "float", "int", "float"]))
champSpellDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "spell1Id", "spell2Id", "spellWinRate", "spellCount", "spellProp"], ["string", "int", "int", "float", "float", "int", "int", "float", "int", "float"]))
vsLaneStatsDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "championIdOpp", "winRateEach", "champCountEach", "champPropEach", "champLevel", "kills", "deaths", "assists", "largestKillingSpree", "largestMultiKill", "firstBloodKill", "firstBloodAssist", "firstTowerKill", "firstTowerAssist", "firstInhibitorKill", "firstInhibitorAssist", "killingSprees", "doubleKills", "tripleKills", "quadraKills", "pentaKills", "turretKills", "inhibitorKills", "longestTimeSpentLiving", "totalTimeCrowdControlDealt", "timeCCingOthers", "totalDamageDealtToChampions", "physicalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "trueDamageDealt", "largestCriticalStrike", "damageDealtToObjectives", "damageDealtToTurrets", "totalHeal", "totalUnitsHealed", "damageSelfMitigated", "totalDamageTaken", "physicalDamageTaken", "magicalDamageTaken", "trueDamageTaken", "visionScore", "wardsPlaced", "wardsKilled", "visionWardsBoughtInGame", "sightWardsBoughtInGame", "goldEarned", "goldSpent", "totalMinionsKilled", "neutralMinionsKilled", "neutralMinionsKilledTeamJungle", "neutralMinionsKilledEnemyJungle", "champLevelOpp", "killsOpp", "deathsOpp", "assistsOpp", "largestKillingSpreeOpp", "largestMultiKillOpp", "firstBloodKillOpp", "firstBloodAssistOpp", "firstTowerKillOpp", "firstTowerAssistOpp", "firstInhibitorKillOpp", "firstInhibitorAssistOpp", "killingSpreesOpp", "doubleKillsOpp", "tripleKillsOpp", "quadraKillsOpp", "pentaKillsOpp", "turretKillsOpp", "inhibitorKillsOpp", "longestTimeSpentLivingOpp", "totalTimeCrowdControlDealtOpp", "timeCCingOthersOpp", "totalDamageDealtToChampionsOpp", "physicalDamageDealtToChampionsOpp", "magicDamageDealtToChampionsOpp", "trueDamageDealtToChampionsOpp", "totalDamageDealtOpp", "magicDamageDealtOpp", "physicalDamageDealtOpp", "trueDamageDealtOpp", "largestCriticalStrikeOpp", "damageDealtToObjectivesOpp", "damageDealtToTurretsOpp", "totalHealOpp", "totalUnitsHealedOpp", "damageSelfMitigatedOpp", "totalDamageTakenOpp", "physicalDamageTakenOpp", "magicalDamageTakenOpp", "trueDamageTakenOpp", "visionScoreOpp", "wardsPlacedOpp", "wardsKilledOpp", "visionWardsBoughtInGameOpp", "sightWardsBoughtInGameOpp", "goldEarnedOpp", "goldSpentOpp", "totalMinionsKilledOpp", "neutralMinionsKilledOpp", "neutralMinionsKilledTeamJungleOpp", "neutralMinionsKilledEnemyJungleOpp", "totalGold10Diff", "xp10Diff", "minionsKilled10Diff", "jungleMinionsKilled10Diff", "totalGold15Diff", "xp15Diff", "minionsKilled15Diff", "jungleMinionsKilled15Diff", "totalGold20Diff", "xp20Diff", "minionsKilled20Diff", "jungleMinionsKilled20Diff", "totalGold25Diff", "xp25Diff", "minionsKilled25Diff", "jungleMinionsKilled25Diff", "totalGold30Diff", "xp30Diff", "minionsKilled30Diff", "jungleMinionsKilled30Diff", "totalGold10SupCount", "xp10SupCount", "minionsKilled10SupCount", "jungleMinionsKilled10SupCount", "totalGold15SupCount", "xp15SupCount", "minionsKilled15SupCount", "jungleMinionsKilled15SupCount", "totalGold20SupCount", "xp20SupCount", "minionsKilled20SupCount", "jungleMinionsKilled20SupCount", "totalGold25SupCount", "xp25SupCount", "minionsKilled25SupCount", "jungleMinionsKilled25SupCount", "totalGold30SupCount", "xp30SupCount", "minionsKilled30SupCount", "jungleMinionsKilled30SupCount", "totalGold10SupProp", "xp10SupProp", "minionsKilled10SupProp", "jungleMinionsKilled10SupProp", "totalGold15SupProp", "xp15SupProp", "minionsKilled15SupProp", "jungleMinionsKilled15SupProp", "totalGold20SupProp", "xp20SupProp", "minionsKilled20SupProp", "jungleMinionsKilled20SupProp", "totalGold25SupProp", "xp25SupProp", "minionsKilled25SupProp", "jungleMinionsKilled25SupProp", "totalGold30SupProp", "xp30SupProp", "minionsKilled30SupProp", "jungleMinionsKilled30SupProp"], ["string", "int", "int", "float", "float", "int", "float", "int", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "int", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float", "float"]))
vsPurchasedDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "championIdOpp", "winRateEach", "champCountEach", "champPropEach", "itemId", "itemWinRateEach", "itemCountEach", "itemPropEach"], ["string", "int", "int", "float", "float", "int", "float", "int", "float", "int", "float", "int", "float"]))
vsRuneDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "championIdOpp", "winRateEach", "champCountEach", "champPropEach", "perk0", "perk1", "perk2", "perk3", "perk4", "perk5", "perkWinRateEach", "perkCountEach", "perkPropEach"], ["string", "int", "int", "float", "float", "int", "float", "int", "float", "int", "int", "int", "int", "int", "int", "float", "int", "float"]))
vsSpellDtype = dict(zip(["laneEdit", "championId", "champCount", "champProp", "winRate", "championIdOpp", "winRateEach", "champCountEach", "champPropEach", "spell1Id", "spell2Id", "spellWinRateEach", "spellCountEach", "spellPropEach"], ["string", "int", "int", "float", "float", "int", "float", "int", "float", "int", "int", "float", "int", "float"]))
outputDtype = dict(champLaneStatsDtype = champLaneStatsDtype, champPurchasedDtype = champPurchasedDtype, champRuneDtype = champRuneDtype, champSpellDtype = champSpellDtype, vsLaneStatsDtype = vsLaneStatsDtype, vsPurchasedDtype = vsPurchasedDtype, vsRuneDtype = vsRuneDtype, vsSpellDtype = vsSpellDtype)

# output formatting
champLaneStatsFormat = dict(zip(["winRate", "champProp", "champLevel", "kills", "deaths", "assists", "largestKillingSpree", "largestMultiKill", "firstBloodKill", "firstBloodAssist", "firstTowerKill", "firstTowerAssist", "firstInhibitorKill", "firstInhibitorAssist", "killingSprees", "doubleKills", "tripleKills", "quadraKills", "pentaKills", "turretKills", "inhibitorKills", "longestTimeSpentLiving", "totalTimeCrowdControlDealt", "timeCCingOthers", "totalDamageDealtToChampions", "physicalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "trueDamageDealt", "largestCriticalStrike", "damageDealtToObjectives", "damageDealtToTurrets", "totalHeal", "totalUnitsHealed", "damageSelfMitigated", "totalDamageTaken", "physicalDamageTaken", "magicalDamageTaken", "trueDamageTaken", "visionScore", "wardsPlaced", "wardsKilled", "visionWardsBoughtInGame", "sightWardsBoughtInGame", "goldEarned", "goldSpent", "totalMinionsKilled", "neutralMinionsKilled", "neutralMinionsKilledTeamJungle", "neutralMinionsKilledEnemyJungle", "totalGold10Diff", "xp10Diff", "minionsKilled10Diff", "jungleMinionsKilled10Diff", "totalGold15Diff", "xp15Diff", "minionsKilled15Diff", "jungleMinionsKilled15Diff", "totalGold20Diff", "xp20Diff", "minionsKilled20Diff", "jungleMinionsKilled20Diff", "totalGold25Diff", "xp25Diff", "minionsKilled25Diff", "jungleMinionsKilled25Diff", "totalGold30Diff", "xp30Diff", "minionsKilled30Diff", "jungleMinionsKilled30Diff", "totalGold10SupProp", "xp10SupProp", "minionsKilled10SupProp", "jungleMinionsKilled10SupProp", "totalGold15SupProp", "xp15SupProp", "minionsKilled15SupProp", "jungleMinionsKilled15SupProp", "totalGold20SupProp", "xp20SupProp", "minionsKilled20SupProp", "jungleMinionsKilled20SupProp", "totalGold25SupProp", "xp25SupProp", "minionsKilled25SupProp", "jungleMinionsKilled25SupProp", "totalGold30SupProp", "xp30SupProp", "minionsKilled30SupProp", "jungleMinionsKilled30SupProp"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
champPurchasedFormat = dict(zip(["champProp", "winRate", "itemWinRate", "itemProp"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
champRuneFormat = dict(zip(["champProp", "winRate", "perkWinRate", "perkProp"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
champSpellFormat = dict(zip(["champProp", "winRate", "spellWinRate", "spellProp"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
vsLaneStatsFormat = dict(zip(["champProp", "winRate", "winRateEach", "champPropEach", "champLevel", "kills", "deaths", "assists", "largestKillingSpree", "largestMultiKill", "firstBloodKill", "firstBloodAssist", "firstTowerKill", "firstTowerAssist", "firstInhibitorKill", "firstInhibitorAssist", "killingSprees", "doubleKills", "tripleKills", "quadraKills", "pentaKills", "turretKills", "inhibitorKills", "longestTimeSpentLiving", "totalTimeCrowdControlDealt", "timeCCingOthers", "totalDamageDealtToChampions", "physicalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "trueDamageDealt", "largestCriticalStrike", "damageDealtToObjectives", "damageDealtToTurrets", "totalHeal", "totalUnitsHealed", "damageSelfMitigated", "totalDamageTaken", "physicalDamageTaken", "magicalDamageTaken", "trueDamageTaken", "visionScore", "wardsPlaced", "wardsKilled", "visionWardsBoughtInGame", "sightWardsBoughtInGame", "goldEarned", "goldSpent", "totalMinionsKilled", "neutralMinionsKilled", "neutralMinionsKilledTeamJungle", "neutralMinionsKilledEnemyJungle", "champLevelOpp", "killsOpp", "deathsOpp", "assistsOpp", "largestKillingSpreeOpp", "largestMultiKillOpp", "firstBloodKillOpp", "firstBloodAssistOpp", "firstTowerKillOpp", "firstTowerAssistOpp", "firstInhibitorKillOpp", "firstInhibitorAssistOpp", "killingSpreesOpp", "doubleKillsOpp", "tripleKillsOpp", "quadraKillsOpp", "pentaKillsOpp", "turretKillsOpp", "inhibitorKillsOpp", "longestTimeSpentLivingOpp", "totalTimeCrowdControlDealtOpp", "timeCCingOthersOpp", "totalDamageDealtToChampionsOpp", "physicalDamageDealtToChampionsOpp", "magicDamageDealtToChampionsOpp", "trueDamageDealtToChampionsOpp", "totalDamageDealtOpp", "magicDamageDealtOpp", "physicalDamageDealtOpp", "trueDamageDealtOpp", "largestCriticalStrikeOpp", "damageDealtToObjectivesOpp", "damageDealtToTurretsOpp", "totalHealOpp", "totalUnitsHealedOpp", "damageSelfMitigatedOpp", "totalDamageTakenOpp", "physicalDamageTakenOpp", "magicalDamageTakenOpp", "trueDamageTakenOpp", "visionScoreOpp", "wardsPlacedOpp", "wardsKilledOpp", "visionWardsBoughtInGameOpp", "sightWardsBoughtInGameOpp", "goldEarnedOpp", "goldSpentOpp", "totalMinionsKilledOpp", "neutralMinionsKilledOpp", "neutralMinionsKilledTeamJungleOpp", "neutralMinionsKilledEnemyJungleOpp", "totalGold10Diff", "xp10Diff", "minionsKilled10Diff", "jungleMinionsKilled10Diff", "totalGold15Diff", "xp15Diff", "minionsKilled15Diff", "jungleMinionsKilled15Diff", "totalGold20Diff", "xp20Diff", "minionsKilled20Diff", "jungleMinionsKilled20Diff", "totalGold25Diff", "xp25Diff", "minionsKilled25Diff", "jungleMinionsKilled25Diff", "totalGold30Diff", "xp30Diff", "minionsKilled30Diff", "jungleMinionsKilled30Diff", "totalGold10SupProp", "xp10SupProp", "minionsKilled10SupProp", "jungleMinionsKilled10SupProp", "totalGold15SupProp", "xp15SupProp", "minionsKilled15SupProp", "jungleMinionsKilled15SupProp", "totalGold20SupProp", "xp20SupProp", "minionsKilled20SupProp", "jungleMinionsKilled20SupProp", "totalGold25SupProp", "xp25SupProp", "minionsKilled25SupProp", "jungleMinionsKilled25SupProp", "totalGold30SupProp", "xp30SupProp", "minionsKilled30SupProp", "jungleMinionsKilled30SupProp"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
vsPurchasedFormat = dict(zip(["champProp", "winRate", "winRateEach", "champPropEach", "itemWinRateEach", "itemPropEach"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
vsRuneFormat = dict(zip(["champProp", "winRate", "winRateEach", "champPropEach", "perkWinRateEach", "perkPropEach"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"])) 
vsSpellFormat = dict(zip(["champProp", "winRate", "winRateEach", "champPropEach", "spellWinRateEach", "spellPropEach"], ["{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}", "{:,.4g}"]))
outputFormat = dict(champLaneStatsFormat = champLaneStatsFormat, champPurchasedFormat = champPurchasedFormat, champRuneFormat = champRuneFormat, champSpellFormat = champSpellFormat, vsLaneStatsFormat = vsLaneStatsFormat, vsPurchasedFormat = vsPurchasedFormat, vsRuneFormat = vsRuneFormat, vsSpellFormat = vsSpellFormat)

formatJson = dict(referenceDtype = referenceDtype, inputDtype = inputDtype, outputDtype = outputDtype, outputFormat = outputFormat)
with open('/home/lj/git/MiniProject_LOL/RefData/formatJson.json', 'w') as savefile:
    json.dump(formatJson, savefile)

# API Key

In [5]:
with open("/home/lj/git/MiniProject_LOL/APIData/product_keys.txt") as f:
    apiList = f.readlines()
apiList = [x.replace("\n", "") for x in apiList]
apikey = apiList[0].replace("\n", "")

In [ ]:
stacks = 0
while True:
    print(stacks)
    if stacks == len(lines)-1:
        stacks -= len(lines)-1
        waiting = 1
        apikey = lines[stacks].replace("\n", "")
        print(apikey)
    else:
        stacks += 1
        waiting = 0
        apikey = lines[stacks].replace("\n", "")
        print(apikey)

In [ ]:
summonerList = pd.DataFrame()
for division in ["I", "II", "III", "IV"]:
    tmp = lol.ibsgpdLeague(apikey, "DIAMOND", division)
    summonerList = pd.concat([summonerList, tmp], axis=0)

# Data Type Json 

In [ ]:
with open('/home/lj/git/KTRolster/RefData/formatJson.json', "r") as loadfile:
    formatJson = json.load(loadfile)

# ddragon 파일 받아오기

In [ ]:
# version data 확인
verRequest = requests.get("https://ddragon.leagueoflegends.com/api/versions.json")
current_version = verRequest.json()[0]
# Champion Info DataFrame
champRequest = requests.get(
    f"http://ddragon.leagueoflegends.com/cdn/{current_version}/data/ko_KR/champion.json"
)
champInfo = pd.DataFrame(champRequest.json())
champInfoDto = (pd.DataFrame(dict(champInfo["data"])).T).reset_index(drop=False)
champInfoDto = champInfoDto.astype(formatJson['referenceDtype']['champInfoDtype'])
# Champion Info DataFrame
itemRequest = requests.get(
    f"http://ddragon.leagueoflegends.com/cdn/{current_version}/data/ko_KR/item.json"
)
itemInfoDto = pd.DataFrame(itemRequest.json()["data"]).T
itemInfoDto.reset_index(drop=False, inplace=True)
itemInfoDto.rename(columns={"index": "itemId"}, inplace=True)
itemInfoDto = itemInfoDto.astype(formatJson['referenceDtype']['itemInfoDtype'])
# Spell Info DataFrame
spellRequest = requests.get(
    f"http://ddragon.leagueoflegends.com/cdn/{current_version}/data/ko_KR/summoner.json"
)
spellInfo = pd.DataFrame(spellRequest.json())
spellInfoDto = pd.concat([spellInfo, pd.DataFrame(dict(spellInfo["data"])).T], axis=1)
spellInfoDto = spellInfoDto.drop(["data"], axis=1)
spellInfoDto = spellInfoDto.reset_index(drop=True)
spellInfoDto = spellInfoDto.astype(formatJson['referenceDtype']['spellInfoDtype'])
# Summoner's Rift DataFrame
mapUrl = f"https://ddragon.leagueoflegends.com/cdn/10.18.1/img/map/map11.png"
lolMap = plt.imread(mapUrl)
# Rune Info DataFrame
runeRequest = requests.get(
    f"http://ddragon.leagueoflegends.com/cdn/{current_version}/data/ko_KR/runesReforged.json"
)
runeInfoDto = pd.DataFrame(runeRequest.json())
perk0 = pd.DataFrame(dict(runeInfoDto["slots"]))
runeInfoDto.drop("slots", axis=1, inplace=True)
for col0 in perk0.columns:
    perk1 = pd.DataFrame(dict(perk0[col0]))
    for col1 in perk1.columns:
        perk2 = pd.DataFrame(dict(perk1[col1])).T
        for col2 in perk2.columns:
            perk3 = pd.DataFrame(dict(perk2[col2])).T
            runeInfoDto = pd.concat([runeInfoDto, perk3], axis=0, ignore_index=True)
runeRequest2 = requests.get(
    f"http://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json"
)
runeInfoDto2 = pd.DataFrame(runeRequest2.json())
runeInfoDto2 = (
    runeInfoDto2[runeInfoDto2.id < 6000]
    .rename(columns={"name": "key", "iconPath": "icon"})
    .drop(["majorChangePatchVersion", "tooltip", "endOfGameStatDescs"], axis=1)
)
runeInfoDto2["name"] = [
    "마법 저항력 +8",
    "방어력 +6",
    "적응형 능력치 +9",
    "체력 +10~90 (레벨에 비례)",
    "재사용 대기시간 감소 +1~10% (레벨에 비례)",
    "공격 속도 +10%",
]
runeInfoDto = pd.concat([runeInfoDto, runeInfoDto2])
runeInfoDto = runeInfoDto.astype(formatJson['referenceDtype']['runeInfoDtype'])
# Item Info DataFrame
itemList = pd.read_csv("/home/lj/git/KTRolster/RefData/itemlist.csv")
itemList = itemList.astype(formatJson['referenceDtype']['itemListDtype'])
legendaryItem = itemList.loc[itemList.category == "전설", "name_Kor"].tolist()
legendaryItem = itemInfoDto[itemInfoDto.name.isin(legendaryItem)].copy()
mythicItem = itemList.loc[itemList.category == "신화", "name_Kor"].tolist()
mythicItem = itemInfoDto[itemInfoDto.name.isin(mythicItem)].copy()

# Data 수집

In [ ]:
LeagueList_cgm = lol.cgmLeague(apikey)
LeagueList_d = lol.ibsgpdLeague(apikey, "DIAMOND", "I")
LeagueList = pd.concat(
    [LeagueList_cgm, LeagueList_d],
    ignore_index=True,
    axis=0
)
today = time.strftime("%Y%m%d", time.gmtime(time.time()))
LeagueList.to_csv(f"~/git/MiniProject_LOL/SummonerList/summonerList_{today}.csv", index=False)

# 챔피언 분류

In [ ]:
MatchReference = pd.read_csv('/home/lj/git/KTRolster/ApiData/MatchReference.csv')
ParticipantIdentity = pd.read_csv('/home/lj/git/KTRolster/ApiData/ParticipantIdentity.csv')
Participant = pd.read_csv('/home/lj/git/KTRolster/ApiData/Participant.csv')
MatchParticipantFrame = pd.read_csv('/home/lj/git/KTRolster/ApiData/MatchParticipantFrame.csv')
MatchEvent = pd.read_csv('/home/lj/git/KTRolster/ApiData/MatchEvent.csv')
itemList = pd.read_csv('/home/lj/git/KTRolster/RefData/itemlist.csv')

MatchReference = MatchReference.astype(formatJson['inputDtype']['MatchReferenceDtype'])
ParticipantIdentity = ParticipantIdentity.astype(formatJson['inputDtype']['ParticipantIdentityDtype'])
Participant = Participant.astype(formatJson['inputDtype']['ParticipantDtype'])
MatchParticipantFrame = MatchParticipantFrame.astype(formatJson['inputDtype']['MatchParticipantFrameDtype'])
MatchEvent = MatchEvent.astype(formatJson['inputDtype']['MatchEventDtype'])

# 산출물

## 데이터 생성

In [ ]:
# Doran 선수 각 gameid, participantid 추출
specificSummonerId = ParticipantIdentity.loc[
    ParticipantIdentity.accountId.isin(["4JPUF_1QjMTx1CyWQ4nVyk278N5iMSnV9AnlbFCWcF6-lhKogzsz7YUF", "gxsNkvhW4I7y8UDgEtrhNUXskkH8m5zy9z-Hu5f_8UM8xsc"]),
    ["gameId", "participantId"]
]
specificSummonerDto = pd.merge(
    Participant, 
    specificSummonerId, 
    how="inner", 
    on=["gameId", "participantId"]
)
for window in [10, 15, 20, 25, 30]:
    tmp = MatchParticipantFrame[
        (MatchParticipantFrame.timestamp >= window * 60 * 1000 - 10000)
        & (MatchParticipantFrame.timestamp <= window * 60 * 1000 + 10000)
    ]
    tmp = tmp[["gameId", "participantId", "totalGold", "xp", "minionsKilled", "jungleMinionsKilled"]]
    tmp[["totalGold", "xp", "minionsKilled", "jungleMinionsKilled"]] = tmp[["totalGold", "xp", "minionsKilled", "jungleMinionsKilled"]].astype(int)
    specificSummonerDto = pd.merge(
        specificSummonerDto, 
        tmp, 
        how="left", 
        on=["gameId", "participantId"]
    )
    specificSummonerDto.rename(
        columns={
            "totalGold": f"totalGold{window}",
            "xp": f"xp{window}",
            "minionsKilled": f"minionsKilled{window}",
            "jungleMinionsKilled": f"jungleMinionsKilled{window}"
        },
        inplace=True,
    )
specificSummonerId = set(
    zip(specificSummonerId.gameId, specificSummonerId.participantId)
)
# Doran 선수 각 game의 블루, 레드팀lane 추출
specificLaneSummonerDto = pd.merge(
    Participant,
    specificSummonerDto[["gameId", "laneEdit"]],
    how="inner",
    on=["gameId", "laneEdit"],
)
specificLaneSummonerId = set(
    zip(specificLaneSummonerDto.gameId, specificLaneSummonerDto.participantId)
)
# Doran 선수 각 game의 상대 lane 추출
opponentSummonerId = specificLaneSummonerId - specificSummonerId
opponentSummonerDto = pd.merge(
    Participant,
    pd.DataFrame(opponentSummonerId, columns=["gameId", "participantId"]),
    how="inner",
    on=["gameId", "participantId"],
)
for window in [10, 15, 20, 25, 30]:
    tmp = MatchParticipantFrame[
        (MatchParticipantFrame.timestamp >= window * 60 * 1000 - 10000)
        & (MatchParticipantFrame.timestamp <= window * 60 * 1000 + 10000)
    ]
    tmp = tmp[["gameId", "participantId", "totalGold", "xp", "minionsKilled", "jungleMinionsKilled"]]
    tmp[["totalGold", "xp", "minionsKilled", "jungleMinionsKilled"]] = tmp[["totalGold", "xp", "minionsKilled", "jungleMinionsKilled"]].astype(int)
    opponentSummonerDto = pd.merge(
        opponentSummonerDto, 
        tmp, 
        how="left", 
        on=["gameId", "participantId"]
    )
    opponentSummonerDto.rename(
        columns={
            "totalGold": f"totalGold{window}",
            "xp": f"xp{window}",
            "minionsKilled": f"minionsKilled{window}",
            "jungleMinionsKilled": f"jungleMinionsKilled{window}",
        },
        inplace=True,
    )
# Doran 선수 각 game의 상대 lane 추출
vsDto = pd.merge(
    specificSummonerDto,
    opponentSummonerDto,
    how="left",
    on=["gameId", "laneEdit"],
    suffixes=("", "Opp"),
)
for i in ["totalGold10","xp10","minionsKilled10","jungleMinionsKilled10","totalGold15","xp15","minionsKilled15","jungleMinionsKilled15","totalGold20","xp20","minionsKilled20","jungleMinionsKilled20","totalGold25","xp25","minionsKilled25","jungleMinionsKilled25","totalGold30","xp30","minionsKilled30","jungleMinionsKilled30"]:
    vsDto[f"{i}Diff"] = vsDto[i] - vsDto[f"{i}Opp"]
    vsDto[f"{i}Sup"] = vsDto[i] - vsDto[f"{i}Opp"] > 0

## 산출물 1-1 - Doran 선수 라인별 챔피언 지표 통계치

In [ ]:
# 산출물 1-1
# Doran 선수 라인별 챔피언 - 빈도
champLaneGroupDto = vsDto.groupby(["laneEdit", "championId"]).agg(
    winRate=("win", "mean"), champCount=("championId", "count")
)
champLaneGroupDto.winRate = champLaneGroupDto.winRate * 100
champLaneCountDto = champLaneGroupDto.reset_index(drop=False).copy()
champLanePerDto = (
    (
        champLaneGroupDto.champCount
        / champLaneGroupDto.groupby("laneEdit").agg(champProp=("champCount", "sum")).champProp
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
champLanePerDto.rename(
    columns={
        0: "champProp"
    }, inplace=True
)
champLaneDto = pd.merge(
    champLaneCountDto, 
    champLanePerDto, 
    how="left",
    on=["laneEdit", "championId"]
)
champLaneDto.sort_values(
    by=["champCount"], inplace=True, ascending=False, ignore_index=True
)
# Doran 선수 라인별 챔피연 - 지표
varList = ["laneEdit", "championId",
           "win", "champLevel", "kills", "deaths", "assists", "largestKillingSpree", "largestMultiKill", "firstBloodKill", "firstBloodAssist", "firstTowerKill", "firstTowerAssist", "firstInhibitorKill", "firstInhibitorAssist", "killingSprees", "doubleKills", "tripleKills", "quadraKills", "pentaKills", "turretKills", "inhibitorKills", "longestTimeSpentLiving", "totalTimeCrowdControlDealt", "timeCCingOthers", 
           "totalDamageDealtToChampions", "physicalDamageDealtToChampions", "magicDamageDealtToChampions", "trueDamageDealtToChampions", "totalDamageDealt", "magicDamageDealt", "physicalDamageDealt", "trueDamageDealt", "largestCriticalStrike", "damageDealtToObjectives", "damageDealtToTurrets", 
           "totalHeal", "totalUnitsHealed", "damageSelfMitigated", "totalDamageTaken", "physicalDamageTaken", "magicalDamageTaken", "trueDamageTaken", 
           "visionScore", "wardsPlaced", "wardsKilled", "visionWardsBoughtInGame", "sightWardsBoughtInGame", 
           "goldEarned", "goldSpent", "totalMinionsKilled", "neutralMinionsKilled", "neutralMinionsKilledTeamJungle", "neutralMinionsKilledEnemyJungle"]
varListVs = varList + [x+"Opp" for x in varList[1:]] + ["totalGold10Diff","xp10Diff","minionsKilled10Diff","jungleMinionsKilled10Diff","totalGold15Diff","xp15Diff","minionsKilled15Diff","jungleMinionsKilled15Diff","totalGold20Diff","xp20Diff","minionsKilled20Diff","jungleMinionsKilled20Diff","totalGold25Diff","xp25Diff","minionsKilled25Diff","jungleMinionsKilled25Diff","totalGold30Diff","xp30Diff","minionsKilled30Diff","jungleMinionsKilled30Diff"]
varlistDiff = ["totalGold10Diff","xp10Diff","minionsKilled10Diff","jungleMinionsKilled10Diff","totalGold15Diff","xp15Diff","minionsKilled15Diff","jungleMinionsKilled15Diff","totalGold20Diff","xp20Diff","minionsKilled20Diff","jungleMinionsKilled20Diff","totalGold25Diff","xp25Diff","minionsKilled25Diff","jungleMinionsKilled25Diff","totalGold30Diff","xp30Diff","minionsKilled30Diff","jungleMinionsKilled30Diff"]
varListSup = ["totalGold10Sup","xp10Sup","minionsKilled10Sup","jungleMinionsKilled10Sup","totalGold15Sup","xp15Sup","minionsKilled15Sup","jungleMinionsKilled15Sup","totalGold20Sup","xp20Sup","minionsKilled20Sup","jungleMinionsKilled20Sup","totalGold25Sup","xp25Sup","minionsKilled25Sup","jungleMinionsKilled25Sup","totalGold30Sup","xp30Sup","minionsKilled30Sup","jungleMinionsKilled30Sup"]
varList += varlistDiff

champLaneGroupDto = (
    vsDto.loc[:, varList]
    .groupby(["laneEdit", "championId"], as_index=False)
    .mean()
    .drop(["win"], axis=1)
)
champLaneStatsDto = pd.merge(
    champLaneDto, 
    champLaneGroupDto, 
    how="left",
    on=["laneEdit", "championId"]
)

champSupGroupDto = vsDto.loc[:, ["laneEdit", "championId"]+varListSup].groupby(["laneEdit", "championId"], as_index=False).sum()
champSupPerDto = vsDto.loc[:, ["laneEdit", "championId"]+varlistDiff].groupby(["laneEdit", "championId"], as_index=False).count()
champSupPerDto.columns = champSupGroupDto.columns
champSupPerDto = champSupGroupDto.iloc[:,2:].divide(champSupPerDto.iloc[:,2:]) * 100
champSupGroupDto.columns = ["laneEdit", "championId"]+[x+'Count' for x in champSupGroupDto.columns.tolist()[2:]]
champSupPerDto.columns = [x+'Prop' for x in champSupPerDto.columns]
champSupDto = pd.concat([champSupGroupDto, champSupPerDto], axis=1)
champLaneStatsDto = pd.merge(
    champLaneStatsDto[champLaneStatsDto.champCount>=3], 
    champSupDto, 
    how="left",
    on=["laneEdit", "championId"]
)
champLaneStatsDto = champLaneStatsDto.astype(formatJson['outputDtype']['champLaneStatsDtype'])

## 산출물 1-2 - Doran 선수 라인별 챔피언 Spell 통계치

In [ ]:
# 산출물 1-2
# Doran 선수 라인별 Spell 선택 빈도 - top 3
champSpellGroupDto = vsDto.groupby(["laneEdit", "championId", "spell1Id", "spell2Id"]).agg(spellWinRate=("win", "mean"), spellCount=("spell1Id", "count"))
champSpellGroupDto.spellWinRate = champSpellGroupDto.spellWinRate * 100
champSpellCountDto = champSpellGroupDto.reset_index(drop=False).copy()
champSpellPerDto = (
    (
        champSpellGroupDto.spellCount
        / champSpellGroupDto.groupby(["laneEdit", "championId"]).agg(spellProp=("spellCount", "sum")).spellProp
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
champSpellPerDto.rename(columns={0: "spellProp"}, inplace=True)
champSpellDto = pd.merge(
    champSpellCountDto,
    champSpellPerDto,
    how="left",
    on=["laneEdit", "championId", "spell1Id", "spell2Id"],
)
champSpellDto = (
    champSpellDto.sort_values(by="spellCount", ascending=False)
    .groupby(["laneEdit", "championId"])
    .head(3)
)
champSpellDto = pd.merge(
    champLaneDto.loc[champLaneDto.champCount>=3],
    champSpellDto,
    how="left",
    on=["laneEdit", "championId"],
)
champSpellDto = champSpellDto.astype(formatJson['outputDtype']['champSpellDtype'])

-> 스펠 사용 빈도 상위 3개로 제한

## 산출물 1-3 - Doran 선수 라인별 챔피언 Rune 통계치

In [ ]:
# 산출물 1-3
# Doran 선수 라인별 Rune 선택
champRuneDto = vsDto.loc[:, ["laneEdit","championId","win","perk0","perk1","perk2","perk3","perk4","perk5"]]

champRuneGroupDto = champRuneDto.groupby(
    ["laneEdit", "championId", "perk0", "perk1", "perk2", "perk3", "perk4", "perk5"]
).agg(perkWinRate=("win", "mean"), perkCount=("perk0", "count"))
champRuneGroupDto.perkWinRate = champRuneGroupDto.perkWinRate * 100
champRuneCountDto = champRuneGroupDto.reset_index(drop=False).copy()
champRunePerDto = (
    (
        champRuneGroupDto.perkCount
        / champRuneGroupDto.groupby(["laneEdit", "championId"]).agg(perkProp=("perkCount", "sum")).perkProp
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
champRunePerDto.rename(columns={0: "perkProp"}, inplace=True)
champRuneDto = pd.merge(
    champRuneCountDto,
    champRunePerDto,
    how="left",
    on=["laneEdit", "championId", "perk0", "perk1", "perk2", "perk3", "perk4", "perk5"],
)
champRuneDto.sort_values(
    by=["perkCount"], inplace=True, ascending=False, ignore_index=True
)
champRuneDto = pd.merge(
    champLaneDto.loc[champLaneDto.champCount>=3],
    champRuneDto,
    how="left",
    on=["laneEdit", "championId"],
)
champRuneDto = champRuneDto.astype(formatJson['outputDtype']['champRuneDtype'])

## 산출물 1-4 - Doran 선수 라인별 챔피언 Item 통계치

In [ ]:
# 산출물 1-4
# Doran 선수 라인별 챔피언 첫 전설 or 신화 아이템
itemList = pd.read_csv("/home/lj/git/KTRolster/RefData/itemlist.csv")
legendaryItem = itemList.loc[itemList.category == "전설", "name_Kor"].tolist()
legendaryItem = itemInfoDto[itemInfoDto.name.isin(legendaryItem)].copy()
mythicItem = itemList.loc[itemList.category == "신화", "name_Kor"].tolist()
mythicItem = itemInfoDto[itemInfoDto.name.isin(mythicItem)].copy()

# doran 선수
champMatchEvent = pd.merge(
    MatchEvent,
    vsDto[["gameId", "participantId"]],
    how="inner",
    on=["gameId", "participantId"],
)
champPurchased = champMatchEvent[
    (champMatchEvent.type == "ITEM_PURCHASED")
    & ((champMatchEvent.itemId.isin(legendaryItem.itemId)) | (champMatchEvent.itemId.isin(mythicItem.itemId)))
].copy()

# champPurchased.loc[[450,625],'itemId']=3087
champPurchased_1st = (
    champPurchased.groupby("gameId").head(1).loc[:, ["gameId", "itemId"]]
)
champPurchased_1st = pd.merge(
    vsDto[["gameId", "laneEdit", "championId", "win"]],
    champPurchased_1st,
    how="left",
    on=["gameId"],
)
champPurchasedGroupDto = champPurchased_1st.groupby(
    ["laneEdit", "championId", "itemId"]
).agg(itemWinRate=("win", "mean"), itemCount=("itemId", "count"))
champPurchasedGroupDto.itemWinRate = champPurchasedGroupDto.itemWinRate * 100
champPurchasedCountDto = champPurchasedGroupDto.reset_index(drop=False).copy()
champPurchasedPerDto = (
    (
        champPurchasedGroupDto.itemCount
        / champPurchasedGroupDto.groupby(["laneEdit", "championId"])
        .agg(itemProp=("itemCount", "sum"))
        .itemProp
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
champPurchasedPerDto.rename(
    columns={
        0: "itemProp"
    }, inplace=True
)
champPurchasedDto = pd.merge(
    champPurchasedCountDto,
    champPurchasedPerDto,
    how="left",
    on=["laneEdit", "championId", "itemId"],
)
champPurchasedDto.sort_values(
    by=["itemCount"], inplace=True, ascending=False, ignore_index=True
)
champPurchasedDto = pd.merge(
    champLaneDto.loc[champLaneDto.champCount>=3],
    champPurchasedDto,
    how="left",
    on=["laneEdit", "championId"],
)
champPurchasedDto = champPurchasedDto.astype(formatJson['outputDtype']['champPurchasedDtype'])

## 산출물 2-1 - Doran 선수 라인별 챔피언별 지표 통계치

In [ ]:
# 산출물 2-1
# Doran 선수 라인별 챔피언별 - 빈도
vsLaneGroupDto = (
    vsDto.loc[:, varListVs]
    .groupby(["laneEdit", "championId", "championIdOpp"])
    .agg(winRateEach=("win", "mean"), champCountEach=("championId", "count"))
)
vsLaneGroupDto.winRateEach = vsLaneGroupDto.winRateEach * 100
vsLaneCountDto = vsLaneGroupDto.reset_index(drop=False).copy()
vsLanePerDto = (
    (
        vsLaneGroupDto.champCountEach
        / vsLaneGroupDto.groupby(["laneEdit", "championId"]).agg(champPropEach=("champCountEach", "sum")).champPropEach
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
vsLanePerDto.rename(
    columns=
    {
        0: "champPropEach"
    }, inplace=True
)
vsLaneDto = pd.merge(
    vsLaneCountDto,
    vsLanePerDto,
    how="left",
    on=["laneEdit", "championId", "championIdOpp"],
)
vsLaneDto.sort_values(
    by="champCountEach", inplace=True, ascending=False, ignore_index=True
)
vsLaneDto = pd.merge(
    champLaneDto,
    vsLaneDto,
    how="left",
    on=["laneEdit", "championId"],
    suffixes=("", "Each"),
)

# Doran 선수 라인별 챔피언별 - 지표
vsGroupDto = (
    vsDto.loc[:, varListVs]
    .groupby(["laneEdit", "championId", "championIdOpp"], as_index=False)
    .mean()
    .drop(["win","winOpp"], axis=1)
)
vsLaneStatsDto = pd.merge(
    vsLaneDto, 
    vsGroupDto, 
    how="left", 
    on=["laneEdit", "championId", "championIdOpp"]
)

vsSupGroupDto = vsDto.loc[:, ["laneEdit", "championId", "championIdOpp"]+varListSup].groupby(["laneEdit", "championId", "championIdOpp"], as_index=False).sum()
vsSupPerDto = vsDto.loc[:, ["laneEdit", "championId", "championIdOpp"]+varlistDiff].groupby(["laneEdit", "championId", "championIdOpp"], as_index=False).count()
vsSupPerDto.columns = vsSupGroupDto.columns
vsSupPerDto = vsSupGroupDto.iloc[:,3:].divide(vsSupPerDto.iloc[:,3:]) * 100
vsSupGroupDto.columns = ["laneEdit", "championId", "championIdOpp"]+[x+'Count' for x in vsSupGroupDto.columns.tolist()[3:]]
vsSupPerDto.columns = [x+'Prop' for x in vsSupPerDto.columns]
vsSupDto = pd.concat([vsSupGroupDto, vsSupPerDto], axis=1)
vsLaneStatsDto = pd.merge(
    vsLaneStatsDto[vsLaneStatsDto.champCount>=3], 
    vsSupDto, 
    how="left",
    on=["laneEdit", "championId", "championIdOpp"]
)
vsLaneStatsDto = vsLaneStatsDto.astype(formatJson['outputDtype']['vsLaneStatsDtype'])

## 산출물 2-2 - Doran 선수 라인별 챔피언별 Spell 통계치 (상대 Spell 고려 x)

In [ ]:
# 산출물 2-2
# Doran 선수 라인별 챔피언별 Spell 선택 빈도 - top 3 -- 상대 Spell 고려 x
vsSpellGroupDto = vsDto.groupby(
    ["laneEdit", "championId", "championIdOpp", "spell1Id", "spell2Id"]
).agg(spellWinRateEach=("win", "mean"), spellCountEach=("spell1Id", "count"))
vsSpellGroupDto.spellWinRateEach = vsSpellGroupDto.spellWinRateEach * 100
vsSpellCountDto = vsSpellGroupDto.reset_index(drop=False).copy()
vsSpellPerDto = (
    (
        vsSpellGroupDto.spellCountEach
        / vsSpellGroupDto.groupby(["laneEdit", "championId", "championIdOpp"]).agg(spellPropEach=("spellCountEach", "sum")).spellPropEach
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
vsSpellPerDto.rename(
    columns={
        0: "spellPropEach"
    }, inplace=True
)
vsSpellDto = pd.merge(
    vsSpellCountDto,
    vsSpellPerDto,
    how="left",
    on=["laneEdit", "championId", "championIdOpp", "spell1Id", "spell2Id"],
)
vsSpellDto.sort_values(
    by="spellCountEach", inplace=True, ascending=False, ignore_index=True
)
vsSpellDto = pd.merge(
    vsLaneDto[vsLaneDto.champCount>=3], 
    vsSpellDto, 
    how="left",
    on=["laneEdit", "championId", "championIdOpp"]
)
vsSpellDto = vsSpellDto.astype(formatJson['outputDtype']['vsSpellDtype'])

## 산출물 2-3 - Doran 선수 라인별 챔피언별 Rune 통계치 (상대 Rune 고려 x)

In [ ]:
# 산출물 2-3
# Doran 선수 라인별 Rune 선택 -- 상대 Rune 고려 x
vsRuneDto = vsDto.loc[:, ["laneEdit","championId","win","championIdOpp","perk0","perk1","perk2","perk3","perk4","perk5"]]
vsRuneGroupDto = vsRuneDto.groupby(
    ["laneEdit", "championId", "championIdOpp", "perk0", "perk1", "perk2", "perk3", "perk4", "perk5"]
).agg(perkWinRateEach=("win", "mean"), perkCountEach=("perk0", "count"))
vsRuneGroupDto.perkWinRateEach = vsRuneGroupDto.perkWinRateEach * 100
vsRuneCountDto = vsRuneGroupDto.reset_index(drop=False).copy()
vsRunePerDto = (
    (
        vsRuneGroupDto.perkCountEach
        / vsRuneGroupDto.groupby(["laneEdit", "championId", "championIdOpp"])
        .agg(perkPropEach=("perkCountEach", "sum")).perkPropEach
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
vsRunePerDto.rename(columns={0: "perkPropEach"}, inplace=True)
vsRuneDto = pd.merge(
    vsRuneCountDto,
    vsRunePerDto,
    how="left",
    on=["laneEdit", "championId", "championIdOpp", "perk0", "perk1", "perk2", "perk3", "perk4", "perk5"]
)
vsRuneDto.sort_values(
    by=["perkCountEach"], inplace=True, ascending=False, ignore_index=True
)
vsRuneDto = pd.merge(
    vsLaneDto[vsLaneDto.champCount>=3], 
    vsRuneDto, 
    how="left",
    on=["laneEdit", "championId", "championIdOpp"]
)
vsRuneDto = vsRuneDto.astype(formatJson['outputDtype']['vsRuneDtype'])

## 산출물 2-4 - Doran 선수 라인별 챔피언별 Item 통계치

In [ ]:
# 맞라인 상대 -- 상대 아이템 고려 x
opponentMatchEvent = pd.merge(
    MatchEvent.astype({"participantId": float}),
    pd.DataFrame(opponentSummonerId, columns=["gameId", "participantId"]),
    how="inner",
    on=["gameId", "participantId"],
)
opponentPurchased = opponentMatchEvent[
    (opponentMatchEvent.type == "ITEM_PURCHASED")
    & (
        (opponentMatchEvent.itemId.isin(legendaryItem.itemId)) | (opponentMatchEvent.itemId.isin(mythicItem.itemId))
    )
].copy()
opponentPurchased_1st = (
    opponentPurchased.groupby("gameId").head(1).loc[:, ["gameId", "itemId"]]
)
opponentPurchased_1st = pd.merge(
    opponentSummonerDto[["gameId", "laneEdit", "championId"]],
    opponentPurchased_1st,
    how="left",
    on=["gameId"],
)
vsPurchased_1st = pd.merge(
    champPurchased_1st,
    opponentPurchased_1st,
    how="inner",
    on=["gameId", "laneEdit"],
    suffixes=["", "Opp"],
)
vsPurchasedGroupDto = vsPurchased_1st.groupby(
    ["laneEdit", "championId", "championIdOpp", "itemId"]
).agg(itemWinRateEach=("win", "mean"), itemCountEach=("itemId", "count"))
vsPurchasedGroupDto.itemWinRateEach = vsPurchasedGroupDto.itemWinRateEach * 100
vsPurchasedCountDto = vsPurchasedGroupDto.reset_index(drop=False).copy()
vsPurchasedPerDto = (
    (
        vsPurchasedGroupDto.itemCountEach
        / vsPurchasedGroupDto.groupby(["laneEdit", "championId", "championIdOpp"]).agg(itemPropEach=("itemCountEach", "sum")).itemPropEach
        * 100
    )
    .reset_index(drop=False)
    .copy()
)
vsPurchasedPerDto.rename(columns={0: "itemPropEach"}, inplace=True)
vsPurchasedDto = pd.merge(
    vsPurchasedCountDto,
    vsPurchasedPerDto,
    how="left",
    on=["laneEdit", "championId", "championIdOpp", "itemId"],
)
vsPurchasedDto.sort_values(
    by=["itemCountEach"], inplace=True, ascending=False, ignore_index=True
)
vsPurchasedDto = pd.merge(
    vsLaneDto[vsLaneDto.champCount>=3], 
    vsPurchasedDto, 
    how="left",
    on=["laneEdit", "championId", "championIdOpp"]
)
vsPurchasedDto = vsPurchasedDto.astype(formatJson['outputDtype']['vsPurchasedDtype'])

## Data 저장

In [ ]:
poc.toCsv(champLaneStatsDto, '/home/lj/git/KTRolster/Output/champLaneStatsDto.csv', globals())
poc.toCsv(champSpellDto, '/home/lj/git/KTRolster/Output/champSpellDto.csv', globals())
poc.toCsv(champRuneDto, '/home/lj/git/KTRolster/Output/champRuneDto.csv', globals())
poc.toCsv(champPurchasedDto, '/home/lj/git/KTRolster/Output/champPurchasedDto.csv', globals())
poc.toCsv(vsLaneStatsDto, '/home/lj/git/KTRolster/Output/vsLaneStatsDto.csv', globals())
poc.toCsv(vsSpellDto, '/home/lj/git/KTRolster/Output/vsSpellDto.csv', globals())
poc.toCsv(vsRuneDto, '/home/lj/git/KTRolster/Output/vsRuneDto.csv', globals())
poc.toCsv(vsPurchasedDto, '/home/lj/git/KTRolster/Output/vsPurchasedDto.csv', globals())

# Map 그려보기

In [ ]:
gameid = 5015588078
positionData = (
    MatchParticipantFrame.loc[MatchParticipantFrame.gameId==gameid, ["participantId", "position_xEdit", "position_yEdit"]]
    .groupby(by=["participantId"])
    .head(10)
    .copy()
)

# TOP
top1x = positionData.loc[(positionData.participantId == 5), "position_xEdit"]
top1y = positionData.loc[(positionData.participantId == 5), "position_yEdit"]
top2x = positionData.loc[(positionData.participantId == 8), "position_xEdit"]
top2y = positionData.loc[(positionData.participantId == 8), "position_yEdit"]
# JUNGLE
jungle1x = positionData.loc[(positionData.participantId == 2), "position_xEdit"]
jungle1y = positionData.loc[(positionData.participantId == 2), "position_yEdit"]
jungle2x = positionData.loc[(positionData.participantId == 7), "position_xEdit"]
jungle2y = positionData.loc[(positionData.participantId == 7), "position_yEdit"]
# MIDDLE
middle1x = positionData.loc[(positionData.participantId == 4), "position_xEdit"]
middle1y = positionData.loc[(positionData.participantId == 4), "position_yEdit"]
middle2x = positionData.loc[(positionData.participantId == 9), "position_xEdit"]
middle2y = positionData.loc[(positionData.participantId == 9), "position_yEdit"]
# BOT
bot1x = positionData.loc[(positionData.participantId == 3), "position_xEdit"]
bot1y = positionData.loc[(positionData.participantId == 3), "position_yEdit"]
bot2x = positionData.loc[(positionData.participantId == 10), "position_xEdit"]
bot2y = positionData.loc[(positionData.participantId == 10), "position_yEdit"]
# SUPPORT
support1x = positionData.loc[(positionData.participantId == 1), "position_xEdit"]
support1y = positionData.loc[(positionData.participantId == 1), "position_yEdit"]
support2x = positionData.loc[(positionData.participantId == 6), "position_xEdit"]
support2y = positionData.loc[(positionData.participantId == 6), "position_yEdit"]


maplink = f"https://ddragon.leagueoflegends.com/cdn/10.18.1/img/map/map11.png"
image = plt.imread(maplink)
plt.imshow(image[::-1], origin="lower")
plt.scatter(top1x, top1y, c="red")
plt.scatter(top2x, top2y, c="salmon")
plt.scatter(jungle1x, jungle1y, c="cyan")
plt.scatter(jungle2x, jungle2y, c="lightblue")
plt.scatter(middle1x, middle1y, c="blue")
plt.scatter(middle2x, middle2y, c="slateblue")
plt.scatter(bot1x, bot1y, c="green")
plt.scatter(bot2x, bot2y, c="lime")
plt.scatter(support1x, support1y, c="yellow")
plt.scatter(support2x, support2y, c="lightyellow")

plt.plot([200, 200, 340, 340, 200], [10, 70, 70, 10, 10], c='yellow')  # bottom
plt.plot([340, 340, 500, 500, 340], [10, 170, 170, 10, 10], c='yellow')  # bottom
plt.plot([440, 440, 500, 500, 460], [170, 310, 310, 170, 170], c='yellow')  # bottom
plt.plot([176, 336, 336, 176, 176], [176, 176, 336, 336, 176])  # mid
plt.plot([10, 10, 70, 70, 10], [200, 340, 340, 200, 200], c='red')  # top
plt.plot([10, 10, 170, 170, 10], [340, 500, 500, 340, 340], c='red')  # top
plt.plot([170, 310, 310, 310, 170], [440, 440, 500, 500, 500], c='red')  # top
plt.show()
Participant.loc[
    Participant.gameId == gameid,
    [
        "participantId",
        "teamId",
        "championId",
        "laneEdit",
        "role",
        "lane",
        "totalMinionsKilled",
        "neutralMinionsKilled",
        "neutralMinionsKilledTeamJungle",
        "neutralMinionsKilledEnemyJungle",
    ],
]

In [ ]:
Matchlist, MatchReference, Match, ParticipantIdentity, TeamStats, Participant, MatchTimeline, MatchFrame, MatchParticipantFrame, MatchEvent = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
for summoner in ['buddha mercy']:
    MatchlistTmp, MatchReferenceTmp, MatchTmp, ParticipantIdentityTmp, TeamStatsTmp, ParticipantTmp, MatchTimelineTmp, MatchFrameTmp, MatchParticipantFrameTmp, MatchEventTmp = poc.laneEdit(apikey, summoner, "2021-02-19 00:00:00", "2021-02-26 00:00:00")
    Matchlist = pd.concat([Matchlist, MatchlistTmp], ignore_index=True)
    MatchReference = pd.concat([MatchReference, MatchReferenceTmp], ignore_index=True)
    Match = pd.concat([Match, MatchTmp], ignore_index=True)
    ParticipantIdentity = pd.concat([ParticipantIdentity, ParticipantIdentityTmp], ignore_index=True)
    TeamStats = pd.concat([TeamStats, TeamStatsTmp], ignore_index=True)
    Participant = pd.concat([Participant, ParticipantTmp], ignore_index=True)
    MatchTimeline = pd.concat([MatchTimeline, MatchTimelineTmp], ignore_index=True)
    MatchFrame = pd.concat([MatchFrame, MatchFrameTmp], ignore_index=True)
    MatchParticipantFrame = pd.concat([MatchParticipantFrame, MatchParticipantFrameTmp], ignore_index=True)
    MatchEvent = pd.concat([MatchEvent, MatchEventTmp], ignore_index=True)